# Exercise - Multi step Workflow - SOLUTION

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field

## 1. Instantiate Chat Model with your API Key

To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key="voc-",
)
```

In [ ]:
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [3]:
logs = []

In [4]:
parser = StrOutputParser()

In [5]:
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [ ]:
idea_prompt = PromptTemplate(
    template=(
        "You are a creative business advisor. "
        "Generate one innovative business idea in the industry: "
        "{industry}. "
        "Provide a brief description of the idea."
    )
)

In [7]:
idea_chain = (
    idea_prompt 
    | llm 
    | parse_and_log_output_chain
)

In [ ]:
idea_result = idea_chain.invoke("agro")

In [9]:
idea_result["output"]

'**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are modular, self-sustaining farming units designed for urban environments and small spaces. Each pod utilizes hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be installed in homes, schools, or community centers, promoting local food production and reducing the carbon footprint associated with transporting produce. Users can connect to a mobile app that provides guidance on crop selection, growth tracking, and even recipes for using their fresh produce. Additionally, the business can offer subscription services for seed kits, nutrients, and maintenance support, creating a recurring revenue model.\n\nThis innovative appr

In [10]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are modular, self-sustaining farming units designed for urban environments and small spaces. Each pod utilizes hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be installed in homes, schools, or community centers, promoting local food production and reducing the carbon footprint associated with transporting produce. Users can connect to a mobile app that provides guidance on crop selection, growth tracking, and even recipes for using their fresh produce. Additionally, the business can offer subscription services for seed kits, nutrients, and maintenance support, creating a recurring revenue model.\n\nT

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [ ]:
analysis_prompt = PromptTemplate(
    template=(
        "Analyze the following business idea: "
        "Idea: {idea} "
        "Identify 3 key strengths and 3 potential weaknesses of the idea."
    )
)

In [ ]:
analysis_chain = (
    analysis_prompt 
    | llm 
    | parse_and_log_output_chain
)

In [ ]:
analysis_result = analysis_chain.invoke(idea_result["output"])

In [14]:
analysis_result["output"]

'### Key Strengths\n\n1. **Sustainability and Environmental Impact:**\n   - The Smart Vertical Farming Pods promote sustainable agriculture by reducing the need for arable land and minimizing water usage through hydroponic and aeroponic systems. This aligns with growing consumer demand for eco-friendly products and practices, potentially attracting environmentally conscious customers.\n\n2. **Urban Food Security:**\n   - By enabling local food production in urban settings, these pods can help address food security issues, particularly in areas with limited access to fresh produce. This can be particularly appealing in densely populated cities where traditional farming is not feasible.\n\n3. **Technological Integration and User Engagement:**\n   - The incorporation of IoT sensors and a mobile app enhances user experience by providing real-time data and personalized guidance. This tech-savvy approach can attract a younger demographic interested in smart home technologies and sustainable 

In [15]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are modular, self-sustaining farming units designed for urban environments and small spaces. Each pod utilizes hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be installed in homes, schools, or community centers, promoting local food production and reducing the carbon footprint associated with transporting produce. Users can connect to a mobile app that provides guidance on crop selection, growth tracking, and even recipes for using their fresh produce. Additionally, the business can offer subscription services for seed kits, nutrients, and maintenance support, creating a recurring revenue model.\n\nT

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [ ]:
report_prompt = PromptTemplate(
    template=(
        "Here is a business analysis: "
        "Strengths & Weaknesses: {output} "
        "Generate a structured business report." 
    )
)

In [17]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [ ]:
report_chain = (
    report_prompt | llm.with_structured_output(schema=AnalysisReport, method="function_calling")
)

In [ ]:
report_result = report_chain.invoke(analysis_result["output"])

In [20]:
report_result

AnalysisReport(strengths=['Sustainability and Environmental Impact: The Smart Vertical Farming Pods promote sustainable agriculture by reducing the need for arable land and minimizing water usage through hydroponic and aeroponic systems, aligning with consumer demand for eco-friendly products.', 'Urban Food Security: These pods enable local food production in urban settings, addressing food security issues in areas with limited access to fresh produce, appealing to densely populated cities.', 'Technological Integration and User Engagement: Incorporating IoT sensors and a mobile app enhances user experience with real-time data and personalized guidance, attracting a younger demographic interested in smart home technologies and sustainable living.'], weaknesses=['High Initial Investment: The cost of developing and manufacturing the Smart Vertical Farming Pods and technology integration may be significant, limiting market entry for price-sensitive consumers.', 'Maintenance and Technical C

## 6. End to End Chain

In [22]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [28]:
e2e_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [ ]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [24]:
e2e_result

AnalysisReport(strengths=['Sustainability and Local Food Production', 'Technological Integration', 'Recurring Revenue Model'], weaknesses=['Initial Cost and Accessibility', 'Technical Complexity', 'Market Competition and Differentiation'])

In [25]:
e2e_result.strengths

['Sustainability and Local Food Production',
 'Technological Integration',
 'Recurring Revenue Model']

In [26]:
e2e_result.weaknesses

['Initial Cost and Accessibility',
 'Technical Complexity',
 'Market Competition and Differentiation']

In [27]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods**\n\n**Description:**\nSmart Vertical Farming Pods are modular, self-sustaining farming units designed for urban environments and small spaces. Each pod utilizes hydroponic and aeroponic systems to grow a variety of crops, from leafy greens to herbs, without the need for soil. Equipped with IoT sensors, these pods monitor and optimize conditions such as light, temperature, humidity, and nutrient levels in real-time, ensuring maximum yield with minimal resource use.\n\nThe pods can be installed in homes, schools, or community centers, promoting local food production and reducing the carbon footprint associated with transporting produce. Users can connect to a mobile app that provides guidance on crop selection, growth tracking, and even recipes for using their fresh produce. Additionally, the business can offer subscription services for seed kits, nutrients, and maintenance support, creating a recurring revenue model.\n\nT

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity